In [4]:
import sqlite3
import csv

# Set a large field size limit manually
csv.field_size_limit(10**8)

# Connect to the SQLite database
conn = sqlite3.connect('snomedct.db')
cursor = conn.cursor()

# Function to load data from a file into a table
def load_data(file_path, table_name):
    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file, delimiter='\t')
        next(reader)  # Skip header row
        rows = [row for row in reader]
        placeholders = ', '.join('?' * len(rows[0]))
        cursor.executemany(f'INSERT OR IGNORE INTO {table_name} VALUES ({placeholders})', rows)
        conn.commit()

# Load concepts data
load_data('C:/Users/wiztu/Downloads/SnomedCT_InternationalRF2_PRODUCTION_20240801T120000Z/SnomedCT_InternationalRF2_PRODUCTION_20240801T120000Z/Snapshot/Terminology/sct2_Concept_Snapshot_INT_20240801.txt', 'Concept')

# Load descriptions data
load_data('C:/Users/wiztu/Downloads/SnomedCT_InternationalRF2_PRODUCTION_20240801T120000Z/SnomedCT_InternationalRF2_PRODUCTION_20240801T120000Z/Snapshot/Terminology/sct2_Description_Snapshot-en_INT_20240801.txt', 'Description')

# Load relationships data
load_data('C:/Users/wiztu/Downloads/SnomedCT_InternationalRF2_PRODUCTION_20240801T120000Z/SnomedCT_InternationalRF2_PRODUCTION_20240801T120000Z/Snapshot/Terminology/sct2_Relationship_Snapshot_INT_20240801.txt', 'Relationship')

# Load stated relationships data
load_data('C:/Users/wiztu/Downloads/SnomedCT_InternationalRF2_PRODUCTION_20240801T120000Z/SnomedCT_InternationalRF2_PRODUCTION_20240801T120000Z/Snapshot/Terminology/sct2_StatedRelationship_Snapshot_INT_20240801.txt', 'StatedRelationship')

# Populate the Description_fts table
cursor.execute("INSERT INTO Description_fts(term) SELECT term FROM Description;")

# Close the connection
conn.close()




In [5]:
import sqlite3
conn = sqlite3.connect('snomedct.db')
cursor = conn.cursor()

# Drop the existing Description_fts table if it exists
cursor.execute('DROP TABLE IF EXISTS Description_fts')

# Create the Description_fts table with an explicit docid
cursor.execute('''
CREATE VIRTUAL TABLE Description_fts USING fts5(
    docid UNINDEXED,  -- Explicitly declare docid
    term
);
''')

# Populate the Description_fts table, ensuring docid matches the id in Description
cursor.execute('''
INSERT INTO Description_fts(docid, term)
SELECT id, term FROM Description;
''')

# Commit changes and close the connection
conn.commit()
conn.close()

In [1]:
from pymedtermino.snomedct import *

In [2]:
# Searching for diabetes
diabetes_results = SNOMEDCT.search("Diabetes")

# Print out results
for concept in diabetes_results:
    print(concept)

SNOMEDCT[127012008]  # Lipoatrophic diabetes (disorder)

SNOMEDCT[982001]  # Diabetes with peripheral circulatory disorders

SNOMEDCT[2751001]  # Fibrocalculous pancreatic diabetes (disorder)

SNOMEDCT[4307007]  # Pregestational diabetes mellitus AND/OR impaired glucose tolerance, modified White class F (disorder)

SNOMEDCT[4783006]  # Maternal diabetes mellitus with hypoglycemia affecting fetus OR newborn (disorder)

SNOMEDCT[5368009]  # Drug-induced diabetes mellitus (disorder)

SNOMEDCT[5969009]  # Diabetes mellitus associated with genetic syndrome (disorder)

SNOMEDCT[8801005]  # Secondary diabetes mellitus (disorder)

SNOMEDCT[9414007]  # Impaired glucose tolerance (disorder)

SNOMEDCT[9859006]  # Insulin-resistant diabetes mellitus AND acanthosis nigricans (disorder)

SNOMEDCT[11530004]  # Brittle diabetes (disorder)

SNOMEDCT[11687002]  # Gestational diabetes mellitus (disorder)

SNOMEDCT[13196008]  # Secondary diabetes insipidus (disorder)

SNOMEDCT[15771004]  # Diabetes insipi

In [7]:
def search_disease(disease_name):
    results = SNOMEDCT.search(disease_name)
    diseases = []
    for concept in results:
        if 'disorder' in concept.term.lower():
            diseases.append(concept)
    return diseases

In [8]:
def get_disease_info(disease_concept):
    info = {
        "name": disease_concept.term,
        "symptoms": [],
        "medications": [],
        "related_info": []
    }

    # Get Symptoms (using has_manifestation or similar)
    if hasattr(disease_concept, "has_manifestation"):
        symptoms = disease_concept.has_manifestation
        info["symptoms"] = [symptom.term for symptom in symptoms]
    elif hasattr(disease_concept, "associated_with"):
        symptoms = disease_concept.associated_with
        info["symptoms"] = [symptom.term for symptom in symptoms]
    
    # Get Medications (using causative_agent or similar)
    if hasattr(disease_concept, "causative_agent"):
        medications = disease_concept.causative_agent
        info["medications"] = [med.term for med in medications]
    elif hasattr(disease_concept, "therapeutic_role"):
        medications = disease_concept.therapeutic_role
        info["medications"] = [med.term for med in medications]
    
    # Get Other Related Information (e.g., pathological_process)
    if hasattr(disease_concept, "associated_morphology"):
        related_info = disease_concept.associated_morphology
        info["related_info"] = [info.term for info in related_info]
    elif hasattr(disease_concept, "pathological_process"):
        related_info = disease_concept.pathological_process
        info["related_info"] = [info.term for info in related_info]
    
    return info

In [9]:
def crawl_diseases(disease_names):
    disease_data = []
    for name in disease_names:
        diseases = search_disease(name)
        for disease in diseases:
            disease_info = get_disease_info(disease)
            disease_data.append(disease_info)
    return disease_data

In [10]:
disease_names = ["Diabetes", "Hypertension", "Asthma", "Migraine", "Tuberculosis"]
disease_data = crawl_diseases(disease_names)

for data in disease_data:
    print(f"Disease: {data['name']}")
    print(f"Symptoms: {', '.join(data['symptoms'])}")
    print(f"Medications: {', '.join(data['medications'])}")
    print(f"Other Info: {', '.join(data['related_info'])}")
    print("-" * 50)

Disease: Lipoatrophic diabetes (disorder)
Symptoms: Skin structure (body structure), Subcutaneous fatty tissue (body structure), Endocrine structure (body structure)
Medications: 
Other Info: Dystrophy (morphologic abnormality), Degeneration (morphologic abnormality)
--------------------------------------------------
Disease: Diabetes with peripheral circulatory disorders
Symptoms: 
Medications: 
Other Info: 
--------------------------------------------------
Disease: Fibrocalculous pancreatic diabetes (disorder)
Symptoms: Endocrine gonad (body structure), Endocrine pancreatic structure (body structure), Digestive structure (body structure), Endocrine structure (body structure)
Medications: 
Other Info: 
--------------------------------------------------
Disease: Pregestational diabetes mellitus AND/OR impaired glucose tolerance, modified White class F (disorder)
Symptoms: Endocrine pancreatic structure (body structure), Endocrine gonad (body structure), Digestive structure (body struc